# Social Insights
> Proyecto realizado para el reto: BBVA Contigo del Hackathon BBVA 2021

## Recolección de datos

In [1]:
# !pip install git+https://github.com/JustAnotherArchivist/snscrape.git

import snscrape.modules.twitter as sntwitter
import pandas as pd

In [2]:
servicios = ["tarjeta de crédito","tarjeta de débito","crédito","cuenta bancaria","nómina","pensión","cuenta de ahorro","seguro","invertir"]
maxExtraer = 5000

In [4]:
df_servicios = []

for s in servicios:
    tweets_list = []

    data = s + ' since:2021-10-22 until:2021-10-23'
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(data).get_items()):
        if i>maxExtraer:
            break
        tweets_list.append([tweet.date, tweet.content, tweet.user.location])

    df_name = "df_" + s
    df_name = pd.DataFrame(tweets_list, columns=['Fecha','Contenido','Ubicacion'])
    df_name["Servicio"]= s

    df_servicios.append(df_name)

In [5]:
df_Twitter = pd.concat(df_servicios)
df_Twitter["Red_Social"] = "Twitter"
df_Twitter.reset_index(drop=True, inplace=True)
df_Twitter

,Fecha,Contenido,Ubicacion,Servicio,Red_Social
0,2021-10-22 23:59:36+00:00,"@AngelMHZ Ah yo nunca tengo, pero mágicamente ...",National District,tarjeta de crédito,Twitter
1,2021-10-22 23:56:57+00:00,#newvideoinmychannel | ¿Cómo manejo mi tarjeta...,Minnesota,tarjeta de crédito,Twitter
2,2021-10-22 23:56:40+00:00,Murió mí billetera\nSucumbió mí tarjeta de cré...,Argentina,tarjeta de crédito,Twitter
3,2021-10-22 23:55:34+00:00,"Si tengo tarjeta de credito de banamex, alguie...","Tampico | Mante, MX",tarjeta de crédito,Twitter
4,2021-10-22 23:54:50+00:00,"sí, también en lo de la tarjeta de crédito.",el abismo,tarjeta de crédito,Twitter
...,...,...,...,...,...
21931,2021-10-22 00:01:02+00:00,@ErickVil5 Eso es verdad. No invierten en tecn...,,invertir,Twitter
21932,2021-10-22 00:00:53+00:00,Aprovecha grandes ofertas y date el gusto de e...,Balderas 144 Col. Centro. CDMX,invertir,Twitter
21933,2021-10-22 00:00:23+00:00,"@adoquin2001 Y no, no puedes ahorrar e inverti...",,invertir,Twitter
21934,2021-10-22 00:00:08+00:00,#ModoIrreverente #JuanDiegoGómez #InvertirMejo...,Colombia,invertir,Twitter


In [6]:
df_Twitter.to_csv('Datasets/df_socialMedia_raw.csv', sep=',', index=False)

## Limpieza y tratamiento

In [7]:
df_socialMedia_clean = df_Twitter.dropna()
filter_ubicacion = df_socialMedia_clean['Ubicacion'].str.lower().str.contains('m.xico|espa.a|colombia')
df_socialMedia_clean = df_socialMedia_clean[filter_ubicacion]
df_socialMedia_clean.head()

,Fecha,Contenido,Ubicacion,Servicio,Red_Social
6,2021-10-22 23:52:52+00:00,@MundarayZair @ClaroColombia En España (soy co...,"Valencia, España",tarjeta de crédito,Twitter
31,2021-10-22 23:00:08+00:00,PAGO POR MEDIO DE PayU para seguridad en tu co...,"Bogotá, Colombia",tarjeta de crédito,Twitter
32,2021-10-22 23:00:01+00:00,Tarjeta de crédito: ¿Cuándo te conviene usarla...,México,tarjeta de crédito,Twitter
34,2021-10-22 22:57:20+00:00,Fui al Citibanamex por una tarjeta de crédito ...,"Tabasco, México",tarjeta de crédito,Twitter
55,2021-10-22 22:28:40+00:00,Me da miedo usar mi tarjeta de crédito 😵‍💫,Mexico,tarjeta de crédito,Twitter


In [8]:
df_socialMedia_clean.to_csv('Datasets/df_socialMedia_clean.csv', sep=',', index=False)